In [1]:
import numpy as np 
import pandas as pd
from keras.models import Model
from keras.layers import Dense, Embedding, Input,GRU
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, Dropout, concatenate
from keras.preprocessing import text, sequence

from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
max_features = 20000
maxlen = 100

In [3]:
import os
import sys
sys.path.append(os.getcwd() + '\\..')
from data.raw_data import *

In [4]:
train = train_df
test = test_df

In [5]:
list_sentences_train = train["comment_text"].fillna("unknown").values

In [6]:
type(list_sentences_train)

numpy.ndarray

In [7]:
list_sentences_train[:5]

array([ "Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
       "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",
       '"\nMore\nI can\'t make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences 

In [11]:
list_sentences_train.shape

(159571,)

In [8]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [9]:
y = train[list_classes].values

In [10]:
type(y)

numpy.ndarray

In [12]:
y.shape

(159571, 6)

In [13]:
list_sentences_test = test["comment_text"].fillna("unknown").values

In [14]:
print(list_sentences_train[0])
y[0]

Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27


array([0, 0, 0, 0, 0, 0], dtype=int64)

In [15]:
tokenizer = text.Tokenizer(num_words=max_features)

In [16]:
tokenizer.fit_on_texts(list(list_sentences_train))

In [17]:
# train data
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
# test data
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [18]:
X_t.shape

(159571, 100)

In [19]:
X_te.shape

(153164, 100)

In [20]:
X_t[:2]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,   688,
           75,     1,   126,   130,   177,    29,   672,  4511, 12052,
         1116,    86,   331,    51,  2278, 11448,    50,  6864,    15,
           60,  2756,   148,     7,  2937,    34,   117,  1221, 15190,
         2825,     4,    45,    59,   244,     1,   365,    31,     1,
           38,    27,   143,    73,  3462,    89,  3085,  4583,  2273,
          985],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     

In [21]:
def cnn_rnn():
    embed_size = 256
    inp = Input(shape=(maxlen, ))
    main = Embedding(max_features, embed_size)(inp)
    main = Dropout(0.2)(main)
    main = Conv1D(filters=32, kernel_size=2, padding='same', activation='relu')(main)
    main = MaxPooling1D(pool_size=2)(main)
    main = Conv1D(filters=32, kernel_size=2, padding='same', activation='relu')(main)
    main = MaxPooling1D(pool_size=2)(main)
    main = GRU(32)(main)
    main = Dense(16, activation="relu")(main)
    main = Dense(6, activation="sigmoid")(main)
    model = Model(inputs=inp, outputs=main)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()     
    return model

In [22]:
model = cnn_rnn()
model.summary()

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 256)          5120000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 32)           16416     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 50, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 50, 32)            2080      
___________________________________________________________

In [23]:
from sklearn.model_selection import train_test_split

print('Positive Labels ')
any_category_positive = np.sum(y,1)
print(pd.value_counts(any_category_positive))
X_t_train, X_t_test, y_train, y_test = train_test_split(X_t, y, 
                                                        test_size = 0.10, 
                                                        )
print('Training:', X_t_train.shape)
print('Testing:', X_t_test.shape)

Positive Labels 
0    143346
1      6360
3      4209
2      3480
4      1760
5       385
6        31
dtype: int64
Training: (143613, 100)
Testing: (15958, 100)


In [24]:
any_category_positive.shape

(159571,)

In [25]:
batch_size = 128 
epochs = 3

file_path="model_best.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)

callbacks_list = [checkpoint, early] #early
model.fit(X_t_train, y_train, 
          validation_data=(X_t_test, y_test),
          batch_size=batch_size, 
          epochs=epochs, 
          shuffle = True,
          callbacks=callbacks_list)

Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143488/143613 [============================>.] - ETA: 0s - loss: 0.0792 - acc: 0.9736
Epoch 00001: val_loss improved from inf to 0.05139, saving model to model_best.h5
143613/143613 [==============================] - 275s 2ms/step - loss: 0.0792 - acc: 0.9736 - val_loss: 0.0514 - val_acc: 0.9818
Epoch 2/3
143488/143613 [============================>.] - ETA: 0s - loss: 0.0452 - acc: 0.9832
Epoch 00002: val_loss improved from 0.05139 to 0.05092, saving model to model_best.h5
143613/143613 [==============================] - 269s 2ms/step - loss: 0.0452 - acc: 0.9832 - val_loss: 0.0509 - val_acc: 0.9816
Epoch 3/3
143488/143613 [============================>.] - ETA: 0s - loss: 0.0394 - acc: 0.9851
Epoch 00003: val_loss did not improve
143613/143613 [==============================] - 270s 2ms/step - loss: 0.0394 - acc: 0.9851 - val_loss: 0.0538 - val_acc: 0.9814


In [26]:
model.save('Whole_model.h5')

In [28]:
model.load_weights(file_path)
y_test = model.predict(X_te)
sample_submission = pd.read_csv(base_dir + "sample_submission.csv")
sample_submission[list_classes] = y_test
sample_submission.to_csv(base_dir + "cnn_gru.csv", index=False)